In [1]:
import sys
sys.path.append('/Users/wavy/Desktop/auto-st/langchain-LLM/')
from langchain.llms.base import LLM
import torch
import transformers 
import models.shared as shared 
from abc import ABC

from langchain.llms.base import LLM
import random
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList
from typing import Optional, List, Dict, Any
from models.loader import LoaderCheckPoint 
from models.base import (BaseAnswer,
                         AnswerResult)



INFO  2023-07-24 18:23:09,860-1d: 
loading model config
llm device: cpu
embedding device: cpu
dir: /Users/wavy/Desktop/auto-st/langchain-LLM
flagging username: 47330f0cb3db41fc9242d0ac9ff50191



In [5]:
import asyncio
from argparse import Namespace
from models.loader.args import parser
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
 
args = parser.parse_args(args=['--model', 'openai-chatgpt-3.5'])

args_dict = vars(args)

shared.loaderCheckPoint = LoaderCheckPoint(args_dict)
torch.cuda.empty_cache()
llm=shared.loaderLLM()

In [11]:
import os
from langchain.llms import OpenAI
os.environ["OPENAI_API_KEY"] = "fk204612-ZsFmjA1waVoinXEZfN6w42Fr7467N5cq"
os.environ["OPENAI_API_BASE"] = "https://oa.api2d.net/v1"
llm = OpenAI(temperature=0)

In [12]:
import sys
from langchain.agents import Tool
from langchain.tools import BaseTool
from agent.custom_search import DeepSearch
from agent.custom_agent import *


tools = [
    Tool.from_function(
        func=DeepSearch.search,
        name="DeepSearch",
        description=""
    )
]
tool_names = [tool.name for tool in tools]
output_parser = CustomOutputParser()
prompt = CustomPromptTemplate(template=agent_template,
                              tools=tools,
                              input_variables=["related_content","tool_name", "input", "intermediate_steps"])

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [16]:
from langchain.agents import BaseSingleActionAgent, AgentOutputParser, LLMSingleActionAgent, AgentExecutor
 

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
print(agent_executor.run(related_content="", input="各省高考分数是多少", tool_name="DeepSearch"))





> Entering new AgentExecutor chain...
<re.Match object; span=(0, 21), match="\nDeepSearch('各省高考分数')">

DeepSearch('各省高考分数')

Observation:2023年高考分数线全部公布，浙江省再夺全国第一，6个原因揭秘: 随着6月26日，西藏自治区招生考试委员会办公室发布《西藏自治区2023年普通高等学校招生录取最低控制分数线》公告，2023年全国内地31个省、市、自治区各层次、各段录取分数线全部浮出水面。
31省份高考分数线全部公布: 高考阅卷工作进入尾声，全国各省份陆续公布高考成绩及各批次各类录取最低控制 ... 随着6月26日西藏2023年高考分数线的新鲜出炉，全国31省份高考分数线均已公布。 山东省教育厅在山东教育卫视召开了山东省2023年普通高考第二次新闻发布会，通报山东省今年 ...
2023各地高考分数线汇总 全国各省市区高考录取最低分数线一览: 通报山东省今年夏季高考评卷工作、分数线划定情况及录取工作安排等。 经省招生考试委员会高考划线工作会议审议批准，现将今年我省高考各类别各批次分数线公布如下： 第一类：夏季高考各类分数线。 1.普通类：特殊类型招生控制线520分，一段线443分 ...
各省高考分数线，差距太大了: 据江苏省教育考试院公布的数据： 2023年，全省物理类680分以上有375人，670分以上1162人，630分以上13586人，600分以上32621人。 而且照目前情况看来，分数会继续下降。低生育率在东北几乎无法解决，东北养个孩子太累了，太累了，太累了。
最低481分，最大相差175分，中央财经大学2023各省录取分数线汇总: 不过，投档分数线不到600分的省、市、自治区，分别是：上海市（上海市的高考总分是660分，投档分数线564分，相当于高考总分750分的641分）、湖南省、湖北省、天津市、浙江省、安徽省、辽宁省、河南省、江西省、浙江省、江苏省、山东省、广东省、北京市，共13个省市。
2022年全国各省高考录取分数线汇总 各省批次线一览: 3、2022年江西高考录取分数线 江西文史、理工类各批次文化录取控制分数线为 ... ②专业成绩最低控制分数线 省统考：美术专业203.00分；音乐专业74.00分；舞蹈专

In [14]:

from langchain.tools import StructuredTool

def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)
# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type. 
agent_executor = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent_executor.run("各省高考分数是多少")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "DeepSearch",
  "action_input": "各省高考分数是多少"
}
```

Observation: 赶紧进来看看！2023各地高考分数线汇总，各省高考分数线是多少？: 天津市2023年普通本科录取控制分数线为472分。 天津市的高考录取模式仍然是“两依据、一参考”，考生的高考总成绩由普通高考和普通高中学业水平等级性考试构成。普通高考和等级考于6月7日至10日进行，约6.9万名考生参加考试。高考各科目采用网上评卷方式 ...
赶紧进来看看！2023各地高考分数线汇总，各省高考分数线是多少？: 天津市2023年普通本科录取控制分数线为472分。 天津市的高考录取模式仍然是“两依据、一参考”，考生的高考总成绩由普通高考和普通高中学业水平等级性考试构成。普通高考和等级考于6月7日至10日进行，约6.9万名考生参加考试。高考各科目采用网上评卷方式 ...
23年各省高考分数线公布，考多少分能上本科？河南考生哭晕在厕所: 23年各省高考分数线公布，考多少分能上本科？ 经过半个月的煎熬，最终各省市的高考分数也陆续揭晓，从整体上看的话，2023年的高考分数线照比2022年的高考录取线均有所下降，其实这也等同于今年是高考“小年”。 目前为止，黑吉辽、内蒙古、上海、安徽 ...
多地公布2023高考分数线，你那里是多少？: 高考阅卷工作进入尾声，全国各省份陆续公布高考成绩及各批次各类录取最低控制分数线。6月23日起，全国多个省份先后“放榜”。 综合官方发布 ...
2023年高考分数线预测，来看看各省市都需要多少分才可以上大学？: 2023年高考分数线预测：各省市录取最低高分线是多少？ 作为即将到来的重要考试，2023年高考的备考早已如火如荼地展开。每一位准备参加高考的考生都希望自己能够取得理想的成绩，顺利地被心仪的大学录取。所以，了解各省市的录取最低高分线成了高考生们 ...
2023年高考分数线汇总 来看看你那里是多少: 现公布如下： 普通类各批次录取控制分数线 一、文科 ... 你所在省份的分数线是多少。 资料图：6月10日下午，北京市2022年高考结束，考生合影留念。
Thought: I know what to re

'2023年各省高考分数线的具体情况可以参考以下链接：https://www.sohu.com/a/460907845_100144590'